## install transformers

In [3]:
! pip install transformers datasets evaluate seqeval accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.2-py3-none-any.whl (7.1 MB)
  Using cached datasets-2.12.0-py3-none-any.whl (474 kB)
  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)
  Using cached seqeval-1.2.2.tar.gz (43 kB)
  Preparing metadata (setup.py) ... done
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)
  Using cached huggingface_hub-0.15.1-py3-none-any.whl (236 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Import libraries

In [4]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence
import random
import pandas as pd
import numpy as np
from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer, AlbertForTokenClassification, TrainingArguments, Trainer
from transformers import AlbertTokenizerFast

from transformers import DataCollatorForTokenClassification




## Load the dataset

In [5]:
data = load_dataset('SpeedOfMagic/ontonotes_english')
data

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/59924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13900 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8262 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/SpeedOfMagic___parquet/SpeedOfMagic--ontonotes_english-955f75f6fda13fd9/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 59924
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13900
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 8262
    })
})

In [6]:
data['train'][0]

{'tokens': ['Sunday', 'the', 'interview', 'with', 'Bob', 'Shapiro', '.'],
 'ner_tags': [15, 0, 0, 0, 1, 2, 0]}

## We are using NER task for token classification

In [7]:
task = "ner"
model_checkpoint = "albert-base-v2"
batch_size = 16

In [8]:
label = data['train'].features[f"{task}_tags"].feature.names
print(label)

['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE']


## just to see how dataset look like

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(data['train'])

,tokens,ner_tags
0,[--],[O]
1,"[I, passed, a, burning, village, and, then, an, emptying, village, ,, people, fleeing, into, the, bush, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2,"[According, to, legend, ,, Liu, Hai, was, the, student, of, a, miserly, Daoist, master, .]","[O, O, O, O, B-PERSON, I-PERSON, O, O, O, O, O, O, B-NORP, O, O]"
3,"[then, we, climbed, everything, on, our, wish, list, on, our, first, try, ,, then, drove, home, without, incident, ..., '', --]","[O, O, O, O, O, O, O, O, O, O, B-ORDINAL, O, O, O, O, O, O, O, O, O, O]"
4,"[On, Friday, ,, stone, -, throwing, Palestinians, confronted, Israeli, troops, protecting, a, Jewish, settlement, in, Gaza, .]","[O, B-DATE, O, O, O, O, B-NORP, O, B-NORP, O, O, O, B-NORP, O, O, B-GPE, O]"
5,"[That, 's, New, York, 's, LaGuardia, Airport, .]","[O, O, B-GPE, I-GPE, I-GPE, B-FAC, I-FAC, O]"
6,"[May, you, be, safe, and, healthy, every, year, ,, my, eminent, brother, ..]","[O, O, O, O, O, O, O, B-DATE, O, O, O, O, O]"
7,"[After, hours, of, conflict, and, debate, ,, that, jury, focuses, on, the, facts, with, near, -, perfect, objectivity, .]","[O, B-TIME, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,"[And, I, still]","[O, O, O]"
9,"[Go, in, not, go, in, or, go, in, with, greater, strength, .]","[O, O, O, O, O, O, O, O, O, O, O, O]"


## Lets preprocess the data

tokenize the data

In [11]:
tokenizer = AlbertTokenizerFast.from_pretrained(model_checkpoint)

In [12]:
# this will make sure the tokenizer is fast
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
## we need to preprocess the data since if our words already tokinze it will tokenize again wchich will end up giving more ids
## we will use truncation = true to reduce the size of the text that are bigger than maximum size and will use
## is_split_into_words=true then we align them
label_all_tokens = True # we can change the value of this flag to change the strategy to set the label for first token only
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], return_offsets_mapping=True, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


In [14]:
## use map to apply the above function on all sentences
tokenize_data = data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/59924 [00:00<?, ? examples/s]

Map:   0%|          | 0/13900 [00:00<?, ? examples/s]

Map:   0%|          | 0/8262 [00:00<?, ? examples/s]

In [15]:
tokenize_data['train'][0]

{'tokens': ['Sunday', 'the', 'interview', 'with', 'Bob', 'Shapiro', '.'],
 'ner_tags': [15, 0, 0, 0, 1, 2, 0],
 'input_ids': [2, 2608, 14, 2421, 29, 1909, 26009, 13, 9, 3],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'offset_mapping': [[0, 0],
  [0, 6],
  [0, 3],
  [0, 9],
  [0, 4],
  [0, 3],
  [0, 7],
  [0, 1],
  [0, 1],
  [0, 0]],
 'labels': [-100, 15, 0, 0, 0, 1, 2, 0, 0, -100]}

## fine tune the model

In [16]:
model = AlbertForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label))

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForTokenClassification: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably

## Initiate training
### before that we need training arguments which will need folder name and other optional

In [17]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

## make all our example to same size 

In [18]:
data_collector = DataCollatorForTokenClassification(tokenizer)

## last we need metric from prediction, we use seqeval metric

In [19]:
metric = load_metric("seqeval")

<ipython-input-19-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")



*   selecting predicted index for each token with max logit
*   cnvert it to string label
*   ignore everywhere we set label of -100 from earliear





In [20]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

accuracy/f1score/recall/precision


In [21]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenize_data["train"],
    eval_dataset=tokenize_data["validation"],
    data_collator=data_collector,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Now we train our trainer

In [23]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.102300,0.130040,0.821149,0.872797,0.846186,0.971947
2,0.073000,0.125604,0.828972,0.870137,0.849056,0.972677
3,0.049300,0.133307,0.820155,0.875973,0.847145,0.972858


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: -100 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 9 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarnin

TrainOutput(global_step=11238, training_loss=0.09137271769765136, metrics={'train_runtime': 2352.701, 'train_samples_per_second': 76.411, 'train_steps_per_second': 4.777, 'total_flos': 487149919187208.0, 'train_loss': 0.09137271769765136, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.13330674171447754,
 'eval_precision': 0.8201549811383866,
 'eval_recall': 0.8759725309622102,
 'eval_f1': 0.8471453112853538,
 'eval_accuracy': 0.9728584171860707,
 'eval_runtime': 76.0444,
 'eval_samples_per_second': 182.788,
 'eval_steps_per_second': 11.428,
 'epoch': 3.0}

In [25]:
predictions, labels, _ = trainer.predict(tokenize_data["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'0': {'precision': 0.8583877995642701,
  'recall': 0.8814317673378076,
  'f1': 0.8697571743929359,
  'number': 447},
 '1': {'precision': 0.8036253776435045,
  'recall': 0.83125,
  'f1': 0.8172043010752688,
  'number': 320},
 '100': {'precision': 0.8328586465234351,
  'recall': 0.8779779387768423,
  'f1': 0.8548233350279614,
  'number': 30642},
 '2': {'precision': 0.7923076923076923,
  'recall': 0.8583333333333333,
  'f1': 0.8239999999999998,
  'number': 360},
 '3': {'precision': 0.6756756756756757,
  'recall': 0.6756756756756757,
  'f1': 0.6756756756756757,
  'number': 74},
 '4': {'precision': 0.6984126984126984,
  'recall': 0.7457627118644068,
  'f1': 0.7213114754098361,
  'number': 118},
 '5': {'precision': 0.8557046979865772,
  'recall': 0.8585858585858586,
  'f1': 0.8571428571428572,
  'number': 594},
 '6': {'precision': 0.8590021691973969,
  'recall': 0.8571428571428571,
  'f1': 0.8580715059588299,
  'number': 462},
 '7': {'precision': 0.8125,
  'recall': 0.8282548476454293,
  'f